In [1]:
import numpy as np
import pandas as pd
import config.config_paths as paths
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Reading the CSV files
cal_data = pd.read_csv(paths.calendar_path)
salestv_data = pd.read_csv(paths.sales_train_val_path)
ss_data = pd.read_csv(paths.sample_submission_path)

In [3]:
timesteps=14
startDay=0

In [4]:
salestv_data.shape

(30490, 1919)

In [5]:
salestv_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [6]:
cal_data.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
ss_data.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
salestv_data.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


In [9]:
salestv_data.isnull()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30486,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30487,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30488,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
print(salestv_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB
None


In [11]:
# Making dates as row to get the data in exact format
salestv_data_transposed = salestv_data.T

In [12]:
salestv_data_transposed.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [13]:
salestv_data_transposed.shape

(1919, 30490)

In [14]:
#remove first 6 rows as we do not need that to feed to our model
salestv_data_cropped = salestv_data_transposed[6+startDay:].astype('float32')
salestv_data_cropped.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,0.0,...,0.0,14.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
d_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,...,0.0,11.0,1.0,0.0,4.0,0.0,0.0,6.0,0.0,0.0
d_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,...,0.0,5.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
d_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,6.0,1.0,0.0,5.0,2.0,0.0,2.0,0.0,0.0
d_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
d_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,...,0.0,13.0,2.0,0.0,7.0,3.0,5.0,4.0,0.0,0.0
d_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,...,0.0,18.0,1.0,0.0,8.0,1.0,0.0,1.0,0.0,0.0
d_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,9.0,0.0,...,0.0,13.0,3.0,0.0,4.0,4.0,1.0,8.0,0.0,0.0


In [15]:
# Initialize a dataframe with zeros for 1969 days in the calendar

daysBeforeEvent1 = pd.DataFrame(np.zeros((cal_data.shape[0], 1)))
daysBeforeEvent2 = pd.DataFrame(np.zeros((cal_data.shape[0], 1)))

snap_CA = cal_data["snap_CA"]
snap_TX = cal_data["snap_TX"]
snap_WI = cal_data["snap_WI"]

In [16]:
# Label 1 to one day before the event_name_1
# Label 1 to one day before the event_name_2
# Sales are likely to increase one day before events like superbowl etc.

for x, y in cal_data.iterrows():
    if not (pd.isnull(cal_data["event_name_1"][x])):
        daysBeforeEvent1[0][x - 1] = 1

    if not (pd.isnull(cal_data["event_name_2"][x])):
        daysBeforeEvent2[0][x - 1] = 1

In [17]:
cal_data.shape

(1969, 14)

In [18]:
# input for training as a feature
daysBeforeEvent1_train = daysBeforeEvent1[startDay:1913].astype('float32')
daysBeforeEvent2_train = daysBeforeEvent2[startDay:1913].astype('float32')


snap_CA_train = snap_CA[startDay:1913].astype('float32')
snap_TX_train = snap_TX[startDay:1913].astype('float32')
snap_WI_train = snap_WI[startDay:1913].astype('float32')

In [19]:
daysBeforeEvent1_test = daysBeforeEvent1[1913:1941].astype('float32')
daysBeforeEvent2_test = daysBeforeEvent2[1913:1941].astype('float32')

snap_CA_test = snap_CA[1913:1941].astype('float32')
snap_TX_test = snap_TX[1913:1941].astype('float32')
snap_WI_test = snap_WI[1913:1941].astype('float32')

In [20]:
print(daysBeforeEvent1_train.shape)
print(daysBeforeEvent2_train.shape)
print(daysBeforeEvent1_test.shape)
print(daysBeforeEvent2_test.shape)

(1913, 1)
(1913, 1)
(28, 1)
(28, 1)


In [21]:
print(snap_CA_train.shape)
print(snap_TX_train.shape)
print(snap_WI_train.shape)
print(snap_CA_test.shape)
print(snap_TX_test.shape)
print(snap_WI_test.shape)

(1913,)
(1913,)
(1913,)
(28,)
(28,)
(28,)


In [22]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent1_train.columns = ["oneDayBeforeEvent1"]
daysBeforeEvent1_train.index = salestv_data_cropped.index

daysBeforeEvent2_train.columns = ["oneDayBeforeEvent2"]
daysBeforeEvent2_train.index = salestv_data_cropped.index

snap_CA_train.columns = ["snap_CA"]
snap_CA_train.index = salestv_data_cropped.index

snap_TX_train.columns = ["snap_TX"]
snap_TX_train.index = salestv_data_cropped.index

snap_WI_train.columns = ["snap_WI"]
snap_WI_train.index = salestv_data_cropped.index

In [23]:
final_data = pd.concat([salestv_data_cropped, daysBeforeEvent1_train, daysBeforeEvent2_train,
                         snap_CA_train, snap_TX_train, snap_WI_train], axis = 1, sort=False)
print(final_data)

          0    1    2    3    4    5    6     7    8    9  ...  30485  30486  \
d_1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  12.0  2.0  0.0  ...    0.0    0.0   
d_2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  15.0  0.0  0.0  ...    0.0    0.0   
d_3     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  7.0  1.0  ...    2.0    0.0   
d_4     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  3.0  0.0  ...    2.0    0.0   
d_5     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...    0.0    0.0   
...     ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...    ...    ...   
d_1909  1.0  1.0  1.0  0.0  1.0  0.0  1.0   4.0  0.0  0.0  ...    0.0    0.0   
d_1910  3.0  0.0  0.0  1.0  2.0  0.0  0.0   6.0  0.0  0.0  ...    1.0    0.0   
d_1911  0.0  0.0  1.0  3.0  2.0  2.0  0.0   3.0  0.0  2.0  ...    0.0    0.0   
d_1912  1.0  0.0  1.0  7.0  2.0  0.0  1.0   2.0  0.0  0.0  ...    0.0    1.0   
d_1913  1.0  0.0  1.0  2.0  4.0  0.0  1.0   1.0  0.0  2.0  ...    1.0    0.0   

        30487  30488  30489  oneDayBefo

In [24]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1913 entries, d_1 to d_1913
Columns: 30495 entries, 0 to snap_WI
dtypes: float32(30495)
memory usage: 222.6+ MB


In [25]:
final_data.shape

(1913, 30495)

In [26]:
# Feature scaling
# Scale the features using MinMaxScaler in range 0-1
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0,1))
final_data.columns = final_data.columns.astype(str)
final_data_scaled = sc.fit_transform(final_data)

print(final_data_scaled)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.16666667 ... 0.         0.         0.        ]
 [0.2        0.         0.16666667 ... 0.         0.         0.        ]
 [0.2        0.         0.16666667 ... 0.         0.         0.        ]]


In [27]:
# creating sequences for LSTM model training
X_train = []
y_train = []

for i in range(timesteps,1913-startDay):
    X_train.append(final_data_scaled[i-timesteps:i])
    y_train.append(final_data_scaled[i][0:30490])
    #İmportant!! if extra features are added (like oneDayBeforeEvent)
    #use only sales values for predictions (we only predict sales)
    #this is why 0:30490 columns are choosen

In [28]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1899, 14, 30495)
(1899, 30490)


In [29]:
# Count NaN values in X_train
nan_count_X_train = np.sum(np.isnan(X_train))
print("Number of NaN values in X_train:", nan_count_X_train)

# Count NaN values in y_train
nan_count_y_train = np.sum(np.isnan(y_train))
print("Number of NaN values in y_train:", nan_count_y_train)

Number of NaN values in X_train: 0
Number of NaN values in y_train: 0


In [31]:
from keras.backend import *
import tensorflow as tf
from keras.callbacks import EarlyStopping

tf.random.set_seed(42)
np.random.seed(42)

n_timesteps = X_train.shape[1] #28
n_products = X_train.shape[2] # all 30490 products + 5 features


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=7,
                           strides=1, padding="causal",
                           activation="relu",
                           input_shape=(n_timesteps, n_products)),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=64, kernel_size=7,
                           strides=1, activation='relu', padding="causal"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30490)
])


opt_adam = tf.keras.optimizers.Adam(clipvalue=0.5)

def rmsse(y_true, y_pred):
    scaling_factor = tf.sqrt(tf.reduce_mean(tf.square(y_true), axis=1))
    error = y_true - y_pred
    rmsse = tf.sqrt(tf.reduce_mean(tf.square(error), axis=1)) / scaling_factor
    return rmsse

model.compile(loss='mean_squared_error',
              optimizer=opt_adam,
              metrics=[rmsse])

model.summary()

# Clear TensorFlow session
tf.keras.backend.clear_session()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
# Training parameters
epochs = 150
batch_size = 100

# Training the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 14, 128)           27323648  
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 128)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 7, 64)             57408     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 3, 64)            0         
 1D)                                                             
                                                                 
 bidirectional (Bidirectiona  (None, 3, 1024)          2363392   
 l)                                                              
                                                        

In [33]:
inputs = final_data[-timesteps:]
inputs = sc.transform(inputs)
print(inputs)

[[0.         0.         0.         ... 0.         1.         1.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]
 [0.         0.         0.16666667 ... 0.         1.         0.        ]
 ...
 [0.         0.         0.16666667 ... 0.         0.         0.        ]
 [0.2        0.         0.16666667 ... 0.         0.         0.        ]
 [0.2        0.         0.16666667 ... 0.         0.         0.        ]]


In [34]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
print(X_test.shape)
predictions = []
testInput = None

for j in range(timesteps,timesteps + 28):
    predicted_sales  = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30495))

    testInput = np.column_stack((np.array(predicted_sales),
                                 daysBeforeEvent1_test.loc[1913 + j - timesteps],
                                 daysBeforeEvent2_test.loc[1913 + j - timesteps],
                                 snap_CA_test.loc[1913 + j - timesteps],
                                 snap_TX_test.loc[1913 + j - timesteps],
                                 snap_WI_test.loc[1913 + j - timesteps]))

    X_test = np.append(X_test, testInput).reshape(1,j + 1,30495)
    predicted_sales = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_sales)

print(testInput.shape)

(1, 14, 30495)
1/1 [==============================] - 0s 42ms/step
(1, 30495)


In [41]:
submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))
submission = submission.T
submission = pd.concat((submission, submission), ignore_index=True)

idColumn = ss_data[['id']]
submission[['id']] = idColumn

cols = list(submission.columns)
cols = cols[-1:]+cols[:-1]
submission = submission[cols]

colsdname = ['id'] + [f'F{i}' for i in range(1,29)]
submission.columns = colsdname

cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
        'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
        'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

submission[cols] = submission[cols].mask(submission[cols] < 0, 0)

submission.to_csv('submission_cnn_lstm.csv', index=False, header=True)

In [ ]:
submission.shape